##### Imports and Utility Functions
Ensure the cell below is runs successfully to include all the helper utilities

In [ ]:
%run workspaceutils

###### Check default lakehouse

In [ ]:
if (mssparkutils.runtime.context['defaultLakehouseId']==None):
    displayHTML('<div style="display: flex; align-items: flex-end;"><img style="float: left; margin-right: 10px;" src="https://github.com/hurtn/images/blob/main/stop.png?raw=true" width="50"><span><h4>Please set a default lakehouse before proceeding</span><img style="float: right; margin-left: 10px;" src="https://github.com/hurtn/images/blob/main/stop.png?raw=true" width="50"></div>')
    print('\n')
    raise noDefaultLakehouseException('No default lakehouse found. Please add a lakehouse to this notebook.')
else: 
    print('Default lakehouse is set to '+ mssparkutils.runtime.context['defaultLakehouseName'] + '('+ mssparkutils.runtime.context['defaultLakehouseId'] + ')')

##### Store metadata of Capacities, Workspaces and Items

In [ ]:
print(updateCp())
print(updateWs())
print(updateItems())
print(updateReports())

##### Save Workspace Git Connection Details

In [ ]:
from pyspark.sql.functions import lit
verbose = False #flag to provide verbose error messages
client = fabric.FabricRestClient()

table_name = 'gitconnections'
spark.sql("Drop table if exists "+ table_name)
dfwks = spark.sql("SELECT distinct ID,Type,Name FROM workspaces where Type!='AdminInsights'").collect()
print("Storing git connection details for all workspaces.\nAny errors will appear below, some you can safely ignore such as the git connection status for the Capacity Metrics workspace...")
for idx,i in enumerate(dfwks):
    if i['Type'] == 'Workspace':
        url = "/v1/workspaces/" + i['ID'] + "/git/connection"
        try:
            print("Storing git details for workspace "+i['Name']+" ("+i['ID']+')...')
            response = client.get(url)

            gitProviderDetailsJSON = response.json()['gitProviderDetails']
            gitConnectionStateJSON = response.json()['gitConnectionState']
            gitSyncDetailsJSON = response.json()['gitSyncDetails']
            df = spark.createDataFrame([i['ID']],"string").toDF("Workspace_ID")
            df=df.withColumn("Workspace_Name",lit(i['Name'])).withColumn("gitConnectionState",lit(gitConnectionStateJSON)).withColumn("gitProviderDetails",lit(json.dumps(gitProviderDetailsJSON))).withColumn("gitSyncDetails",lit(json.dumps(gitSyncDetailsJSON)))

            if idx == 0:
                dfall = df
            else:
                dfall= dfall.union(df)
        except Exception as error:
            errmsg =  "Couldn't get git connection status for workspace " + i['Name'] + "("+ i['ID'] + ")."
            if (verbose):
                 errmsg = errmsg + "Error: "+str(error)
            print(str(errmsg))

dfall.withColumn("metaTimestamp",current_timestamp()).write.mode('overwrite').option("mergeSchema", "true").saveAsTable(table_name)
print('Done')



In [ ]:
mssparkutils.runtime.context

In [ ]:
df = spark.sql("SELECT * FROM gitbranching.gitconnections where workspace_id = '"+ mssparkutils.runtime.context['currentWorkspaceId'] + "'")

gitsql = "select gt.gitConnectionState,gt.gitProviderDetails, wks.name Workspace_Name, wks.id Workspace_ID from gitconnections gt " \
         "inner join workspaces wks on gt.Workspace_Name = wks.name " \
         "where gt.gitConnectionState = 'ConnectedAndInitialized' and wks.id = '"+mssparkutils.runtime.context['currentWorkspaceId']+"'" 

dfgitdetails = spark.sql(gitsql).collect()
display(dfgitdetails)

In [ ]:
df = spark.sql("SELECT * FROM gitbranching.capacities LIMIT 1000")
display(df)

In [ ]:

df = spark.sql("SELECT capacity_id,name FROM gitbranching.workspaces where id = '"+ mssparkutils.runtime.context['currentWorkspaceId'] + "'").collect()
current_capacity_id = df[0][0]
current_ws_name = df[0][1]
print('Current capacity ID = '+current_capacity_id + ' for workspace ' + current_ws_name)

In [ ]:
branch_to_new_ws = 'DEV_Custom_Feature_02'
branch_name = 'DEV_Custom_Feature_02_branch'
try:
    print("Creating workspace: " + branch_to_new_ws + " in capacity "+ current_capacity_id +"...")
    response = fabric.create_workspace(branch_to_new_ws,current_capacity_id) 
    new_workspace_id = response
    print("Created workspace with ID: " + new_workspace_id)
except Exception as error:
    errmsg =  "Failed to recreate workspace " +branch_to_new_ws + " with capacity ID ("+ current_capacity_id + ") due to: "+str(error)
    print(errmsg)

# Now update workspace table
updateWs()

In [ ]:
df = spark.sql("SELECT * FROM gitbranching.workspaces where name = '"+ branch_to_new_ws + "'")
display(df)

In [ ]:
gitpayloadstr= '{"gitProviderDetails": ' + dfgitdetails[0][1] + '}'
print("Before: " + gitpayloadstr)
gitpayload = json.loads(gitpayloadstr)
gitpayload["gitProviderDetails"]["branchName"] = branch_name
print("After: " +json.dumps(gitpayload))

In [ ]:
url = "/v1/workspaces/" +  new_workspace_id + "/git/connect"

try:
    print('Attempting to connect workspace '+ branch_to_new_ws)
    response = client.post(url,json= gitpayload)
    print(str(response.status_code) + response.text) 
    success = True
    
except Exception as error:
    errmsg =  "Couldn't connect git to workspace " + branch_to_new_ws + ". Error: "+str(error)
    print(str(errmsg))
    success = False
# If connection successful then try to initialise    
if (success):
    url = "/v1/workspaces/" + new_workspace_id + "/git//initializeConnection"
    payload = {"initializationStrategy":"PreferRemote"}
    try:
        print('Attempting to initialize git connection for workspace '+ branch_to_new_ws)
        response = client.post(url,json= payload)
        #print(str(response.status_code) + response.text) 
        commithash = response.json()['remoteCommitHash']
        print('Successfully initialized. Updating with commithash '+commithash)
        if commithash!='':
            url = "/v1/workspaces/" + new_workspace_id + "/git/updateFromGit"
            payload = '{"remoteCommitHash": "' + commithash + '","conflictResolution": {"conflictResolutionType": "Workspace","conflictResolutionPolicy": "PreferWorkspace"},"options": {"allowOverrideItems": true}}'
            response = client.post(url,json= json.loads(payload))
            print(str(response.status_code))
    except Exception as error:
        errmsg =  "Couldn't initialize git for workspace " +branch_to_new_ws + ". Error: "+str(error)
        print(str(errmsg))

In [ ]:
df = spark.sql("SELECT * FROM gitbranching.workspaces LIMIT 1000")
display(df)